In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import re  
#
# CSV-Datei mit Semikolon als Trennzeichen lesen
df = pd.read_csv('data.csv', sep=';')
df.rename(columns=lambda x: re.sub(r'\W+', ' ', x).strip(), inplace=True)
# Die ersten fünf Zeilen anzeigen
df.head()
df.shape

# Überprüfen, welche Werte NaN sind (True = NaN, False = kein NaN)
print(df.isna().sum().sum())
#
# Überprüfung aller fehlenden Werte in einem Befehl
missing_values = df.isnull() | df.eq('') | df.eq('None') | df.eq('null') | df.eq('?') | df.eq('-') | df.eq('N/A') | df.eq('unknown')
# Anzahl der fehlenden Werte pro Spalte
missing_values.sum()
# Summe aller fehlenden Werte im gesamten DataFrame
missing_values.sum().sum()
# Zeige alle Zeilen, die mindestens einen fehlenden Wert haben
df[missing_values.any(axis=1)]
#########################################################################################################################################
#df.info()
#df.describe()
print(df.columns)
df.head(10)

0
Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime evening attendance', 'Previous qualification',
       'Previous qualification grade', 'Nacionality', 'Mother s qualification',
       'Father s qualification', 'Mother s occupation', 'Father s occupation',
       'Admission grade', 'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem credited',
       'Curricular units 1st sem enrolled',
       'Curricular units 1st sem evaluations',
       'Curricular units 1st sem approved', 'Curricular units 1st sem grade',
       'Curricular units 1st sem without evaluations',
       'Curricular units 2nd sem credited',
       'Curricular units 2nd sem enrolled',
       'Curricular units 2nd sem evaluations',
       'Curricular units 2nd sem approved', 'Curricular units 2nd sem grade',
       'Curricular units 2nd s

,Marital status,Application mode,Application order,Course,Daytime evening attendance,Previous qualification,Previous qualification grade,Nacionality,Mother s qualification,Father s qualification,...,Curricular units 2nd sem credited,Curricular units 2nd sem enrolled,Curricular units 2nd sem evaluations,Curricular units 2nd sem approved,Curricular units 2nd sem grade,Curricular units 2nd sem without evaluations,Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate
5,2,39,1,9991,0,19,133.1,1,37,37,...,0,5,17,5,11.500000,5,16.2,0.3,-0.92,Graduate
6,1,1,1,9500,1,1,142.0,1,19,38,...,0,8,8,8,14.345000,0,15.5,2.8,-4.06,Graduate
7,1,18,4,9254,1,1,119.0,1,37,37,...,0,5,5,0,0.000000,0,15.5,2.8,-4.06,Dropout
8,1,1,3,9238,1,1,137.0,62,1,1,...,0,6,7,6,14.142857,0,16.2,0.3,-0.92,Graduate
9,1,1,1,9238,1,1,138.0,1,1,19,...,0,6,14,2,13.500000,0,8.9,1.4,3.51,Dropout
